## Import các thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Thu thập dữ liệu

- Nhóm em thu thập dữ liệu về **chuyển nhượng cầu thủ của các đội bóng ở 5 giải vô địch bóng đá nam hàng đầu của Châu Âu từ cuồi mùa giải năm 2014 trở về sau**. Dữ liệu lấy từ trang `Kaggle`, file dữ liệu là 'Transfer_data.csv', gốc file dữ liệu là <a href='https://www.kaggle.com/ogrofratesi/football-data?select=Transfer_data.csv&fbclid=IwAR1RxKLlrIlNG9DkyC6DjadHB4zaXjDxSaNkRS_O9c2UR1dCrp-_4_mmvq8'>ở đây</a>
- Dữ liệu này là dữ liệu dạng public nên được cho phép người khác có thể lấy và sử dụng. License của dữ liệu là `CCO: Public Domain`.

---

## Khám phá dữ liệu

### Đọc dữ liệu từ file csv vào dataframe

In [2]:
transfer_df = pd.read_csv('./data/Transfer_data.csv', encoding='utf-8')
transfer_df.head()

,Unnamed: 0,Name,Position,Age,Team_from,League_from,Team_to,League_to,Season,Market_value,Transfer
0,3467,Luis SuÃ¡rez,Centre-Forward,27,Liverpool,EPL,Barcelona,La_liga,2014,52.0,81.72
1,3468,James RodrÃ­guez,Attacking Midfield,23,Monaco,Ligue_1,Real Madrid,La_liga,2014,60.0,75.00
2,3469,Ãngel Di MarÃ­a,Right Winger,26,Real Madrid,La_liga,Manchester United,EPL,2014,50.0,75.00
3,3470,David Luiz,Centre-Back,27,Chelsea,EPL,Paris Saint Germain,Ligue_1,2014,26.0,49.50
4,3471,Alexis SÃ¡nchez,Left Winger,25,Barcelona,La_liga,Arsenal,EPL,2014,30.0,42.50


### Dữ liệu gồm bao nhiêu dòng và cột ?

In [3]:
num_rows, num_cols = transfer_df.shape
print('Number of rows: {}\nNumber of columns: {}'.format(num_rows, num_cols))

Number of rows: 934
Number of columns: 11


### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Theo mô tả và quan sát sơ bộ qua tập dữ liệu thì mỗi dòng cho biết thông tin chuyển nhượng của một cầu thủ. Không có dòng nào có dấu hiệu khác nhau so với cá dòng còn lại 

### Dữ liệu các dòng có bị lặp không ?

In [4]:
is_duplicated_row = False if len(transfer_df.drop_duplicates()) == transfer_df.shape[0] else True
is_duplicated_row

False

### Mỗi cột có ý nghĩa gì ?

Dưới đây là phần mô tả của các cột dữ liệu trong file `Transfer_data.csv` trên Kaggle:
- **Unnamed: 0**: Không xác định
- **Name**: Tên cầu thủ chuyển nhượng
- **Position**: Vị trí thi đấu của cầu thủ
- **Age**: Tuổi cầu thủ
- **Team_from**: Tên câu lạc bộ đội bóng mà cầu thủ đó thi đấu trước khi chuyển nhượng
- **League_from**: Tên giải đấu của câu lạc bộ cầu thủ đó thi đấu trước khi chuyển nhượng
- **Team_to**: Tên câu lạc bộ đội bóng mà cầu thủ đó chuyển nhượng đến
- **League_to**: Tên giải đấu của câu lạc bộ cầu thủ đó chuyển nhượng đến
- **Season**: Mùa giải chuyển nhượng
- **Market_value**: Giá trị của cầu thủ trên thị trường chuyển nhượng (tính theo đơn vị triệu euro)
- **Transfer**: Giá trị chuyển nhượng thực tế mà câu lạc bộ mới phải trả để sở hữu cầu thủ đó (tính theo đơn vị triệu euro)

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

Lấy `dtype` là kiểu dữ liệu của mỗi cột trong dữ liệu và lưu kết quả vào series col_dtypes, series này có index là tên các cột 

In [5]:
col_dtypes = pd.Series(transfer_df.dtypes, transfer_df.columns)
col_dtypes

Unnamed: 0        int64
Name             object
Position         object
Age               int64
Team_from        object
League_from      object
Team_to          object
League_to        object
Season            int64
Market_value    float64
Transfer        float64
dtype: object

Ta cần kiểm tra các cột có kiểu dữ liệu là `object` vì kiểu dữ liệu này có thể bao gồm nhiều kiểu dữ liệu khác nhau trong nó.<br>
Viết 1 hàm có đầu vào là 1 cột (series) có dtype là `objject` và trả về kiểu dữ liệu thật sự của cột dữ liệu đó

In [6]:
def open_dtype_object(s):
    dtypes = set()
    s.apply(lambda x: dtypes.add(type(x)))
    return dtypes

In [7]:
# Kiêm tra kết quả của từng cột có dtype là object
col_names = ['Name', 'Position', 'Team_from', 'League_from', 'Team_to', 'League_to']
for col in col_names:
    print(f'{col} --> {open_dtype_object(transfer_df[col])}')

Name --> {<class 'str'>}
Position --> {<class 'str'>}
Team_from --> {<class 'str'>}
League_from --> {<class 'str'>}
Team_to --> {<class 'str'>}
League_to --> {<class 'str'>}


Ta có thể thấy kiểu dữ liệu của các cột trên đều là `str` nên hợp lí  

**Các vấn đề cần tiền xử lý:**
- Xóa cột `Unnamed: 0` vì cột này không có ý nghĩa gì để khám phá

## Tiền xử lý dữ liệu

### Xóa cột `Unnamed: 0` trong dữ liệu

In [8]:
transfer_df.drop(columns=['Unnamed: 0'], inplace=True)

## Khám phá dữ liệu (tiếp tục)

### Với mỗi cột có kiểu dữ liệu dạng số (numerical), các giá trị được phân bố như thế nào ?

Ta có các cột có kiểu dữ liệu numeric là: `Age`, `Season`, `Market_value`, `Transfer`<br>
<br>Với mỗi cột numeric em sẽ tính tỉ lệ % giá trị thiếu (từ 0 đến 100), min, max. Sau đó em lưu kết quả vào dataframe `nume_col_profiles_df`. Dataframe này có 3 dòng là "missing_ratio", "min", "max"; và có 4 cột tương ứng với 4 cột ở trên 

In [9]:
def missing_ratio(df):
    return df.isna().mean()*100

nume_cols = ['Age', 'Season', 'Market_value', 'Transfer']
nume_col_profiles_df = transfer_df[nume_cols].agg([missing_ratio, min, max])
nume_col_profiles_df

,Age,Season,Market_value,Transfer
missing_ratio,0.0,0.0,0.535332,0.0
min,16.0,2014.0,0.150000,4.0
max,33.0,2018.0,120.000000,222.0


Các cột trên có min và max có lẽ là không có gì bất thường

### Với mỗi cột có kiểu dữ liệu dạng phân loại (categorical), các giá trị được phân bố như thế nào ?


Dataframe đang xử lý có các cột dạng categorical là: `Name`, `Position`, `Team_from`, `League_from`, `Team_to`, `League_to`<br>
<br>Với mỗi cột categorical, em sẽ tính tỉ lệ % giá trị thiếu (từ 0 đến 100), số lượng giá trị khác nhau (không xét giá trị thiếu), list các giá trị khác nhau (lấy 5 mẫu) (không xét giá trị thiếu). Sau đó em lưu kết quả vào dataframe `cate_col_profiles_df`. Dataframe này có 3 dòng là "missing_ratio", "num_diff_vals", "diff_vals"; và gồm 6 cột tương ứng ở trên

In [10]:
def missing_ratio(df):
    return df.isna().mean()*100
def num_diff_vals(df):
    return len(df.dropna().unique())
def diff_vals(df):
    return df.dropna().unique()[:5]

cate_cols = ['Name', 'Position', 'Team_from', 'League_from', 'Team_to', 'League_to']
cate_col_profiles_df = transfer_df[cate_cols].agg([missing_ratio, num_diff_vals, diff_vals])
cate_col_profiles_df

,Name,Position,Team_from,League_from,Team_to,League_to
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0
num_diff_vals,789,13,222,37,91,5
diff_vals,"[Luis SuÃ¡rez, James RodrÃ­guez, Ãngel Di Mar...","[Centre-Forward, Attacking Midfield, Right Win...","[Liverpool, Monaco, Real Madrid, Chelsea, Barc...","[EPL, Ligue_1, La_liga, Serie_A, Liga NOS]","[Barcelona, Real Madrid, Manchester United, Pa...","[La_liga, EPL, Ligue_1, Bundesliga, Serie_A]"


Nhìn chung không có gì bất thường ở các cột dữ liệu này